In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import io
import numpy as np
import pandas as pd
from zipfile import ZipFile
from urllib.request import urlopen
from pandas.api.types import is_numeric_dtype
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
path = '/content/sample_data/' 
train_file = ZipFile(path + 'train.csv.zip')
train = pd.read_csv(train_file.open('train.csv'))
macro_file = ZipFile(path + 'macro.csv.zip')
macro = pd.read_csv(macro_file.open('macro.csv'))
train = train.merge(macro, on='timestamp', how='inner')

In [ ]:
# проверием наличие столбцов без единого значения
for col in train:
    if train[col].isna().sum() == train.shape[0]:
        print(col)

feature_columns = [col for col in train.columns if col not in ['id', 'timestamp', 'price_doc']]

# удаляем дубликаты строк
train.drop_duplicates(subset=feature_columns, inplace=True)

# разделим количественные и категориальные признаки
cat_features=[]
for col in feature_columns:
    if not is_numeric_dtype(train[col]):
        cat_features.append(col)      
num_features = []
for col in feature_columns:
    if col not in cat_features:
        num_features.append(col)
len(cat_features), len(num_features)

# категориальные признаки, где есть пропуски
for col in cat_features:
    if train[col].isna().sum().any():
        print(col)

# выводим распределение по всем категориальным признакам
# последние три не являются категориальными признаками
for col in cat_features:
    print(train[col].value_counts())
    print('--'*25)

In [ ]:
def get_number(value):
    """Функция для преобразования строки в число
       Если такое преобразование невозможно, то возвращаем None"""
    try:
        return float(value.replace(',', '.'))
    except Exception as ex:
        return None

In [ ]:
# преобразуем згачения трех стлобцов в числовые
train['child_on_acc_pre_school'] = train['child_on_acc_pre_school'].apply(get_number)
train['modern_education_share'] = train['modern_education_share'].apply(get_number)
train['old_education_build_share'] = train['old_education_build_share'].apply(get_number)

In [ ]:
# уберем эти три признака с категориальных фичей и добавим в количественные
num_features = num_features + cat_features[-3:]
cat_features = cat_features[:-3]

# смотрим пропуски в категориальных фичах
# пропусков нет 
train[cat_features].info()

In [ ]:
# категориальные признаки превращаем в количественные
for col in cat_features:
    train[col] = train[col].astype('category').cat.codes

In [ ]:
# заполняем пропуски медианой в количественных признаках
for col in num_features:
    train[col] = train[col].fillna(train[col].median())

In [ ]:
# график ящик с усами признака children_preschool
sns.boxplot(train['children_preschool'])

In [ ]:
# график ящик с усами признака num_room
sns.boxplot(train['num_room'])

In [ ]:
# только по двум графикам можно определить наличие выбросов (аномальных значений)
# заменим аномальные значения медианой 
def replace_outliers(df, column_name):
    median = df[column_name].median()
    std = df[column_name].std()
    outliers = ((df[column_name] - median).abs()) > 3*std
    df[outliers][column_name] = np.nan
    df[column_name].fillna(median, inplace=True)

for col in num_features:
    replace_outliers(train, col)

In [ ]:
# смотрим на распределение целевой переменной

fig, ax = plt.subplots(figsize=(10,5))
sns.distplot(train['price_doc'].values, bins=100, kde=True, ax=ax)
plt.xlabel('price_doc', fontsize=12)
plt.show()

In [ ]:
# так как распределение не соотвествует нормальному, применяем логарифмирование
# после логарифмирования распеределение стала близка к нормальному
train['price_doc_log'] = np.log(train['price_doc'])
fig, ax = plt.subplots(figsize=(10,5))
sns.distplot(train['price_doc_log'].values, bins=100, kde=True, ax=ax)
plt.xlabel('log(price_doc)', fontsize=12)
plt.show()

In [ ]:
# смотрим на корреляцию признаков с целевой переменной

pearson = train[feature_columns + ['price_doc_log']].corr(method='pearson')
correlations = pearson['price_doc_log'][:-1]
correlations[abs(correlations).argsort()[::-1]]

In [ ]:
# у многих признаков очень мало корреляции с целевой переменной
# удаляем те признаки, где корреляция меньше 0.1
correlations = correlations[correlations >= 0.1]

In [ ]:
# график корреляции признаков с целевой переменной
cols_to_visualize = correlations[abs(correlations).argsort()[::-1]].index.values.tolist()\
                                                                        + ['price_doc_log']
plt.figure(figsize=(20, 20))
sns.heatmap(train[cols_to_visualize].corr(), cmap='viridis',
            xticklabels=train[cols_to_visualize].corr().columns.values,
            yticklabels=train[cols_to_visualize].corr().columns.values)
plt.title("График матрицы корреляции", fontsize=20)

In [ ]:
# список признаков для обучения
features = list(correlations.index)
print(features)

In [ ]:
X, y = train[[col for col in list(train.columns) if 'price_doc' not in col]], train['price_doc_log']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=23, shuffle=True)

In [ ]:
# обучаем линейную регрессию
model = LinearRegression()
model.fit(X_train[features], y_train)

# получаем важность признаков
importance = model.coef_

# объеднияем важность признаков с названиями столбцов
importance = list(zip(list(X_train[features].columns), importance))
# пролучаем важность признаков с положительным знаком и сортируем
positive = [feature for feature in importance if feature[1] >= 0]
positive = sorted(positive, key=lambda x: x[1], reverse=True)
positive = pd.DataFrame(positive, columns=['feature', 'score'])
# пролучаем важность признаков с отрицательным знаком и сортируем
negative = [feature for feature in importance if feature[1] < 0]
negative = sorted(negative, key=lambda x: x[1])
negative = pd.DataFrame(negative, columns=['feature', 'score'])

# для удобства создаем новый столбец признака с его номером
positive['feature_num'] = positive['feature'] + '_' + list(map(str, list(positive.index.values))) 
negative['feature_num'] = negative['feature'] + '_' + list(map(str, list(negative.index.values))) 

In [ ]:
#  рисуем графики важности признаков
plt.figure(figsize=(10, 10))
sns.barplot(y='feature_num', x='score', data=positive, orient='h')
plt.title('Важность признаков', fontsize=18)

plt.figure(figsize=(10, 10))
sns.barplot(y='feature_num', x='score', data=negative, orient='h')
plt.title('Важность признаков', fontsize=18)

In [ ]:
# по первому графику отбираем 13 самых важных признаков, из второго 12
most_important_features = positive['feature'].loc[:13].values.tolist() + negative['feature'].loc[:12].values.tolist()

In [ ]:
# выводим показатели качества модели на обучающей выборке
lr = LinearRegression()
print('MAE: ',
      np.array(cross_val_score(lr, X_train[most_important_features], 
                               y_train, scoring='neg_mean_absolute_error', cv=5)).mean())
print('MAPE: ', 
      np.array(cross_val_score(lr, X_train[most_important_features],
                               y_train, scoring='neg_mean_absolute_percentage_error', cv=5)).mean())
lr.fit(X_train[most_important_features], y_train)

MAE:  -0.36414449616716477
MAPE:  -0.023816098169988635


LinearRegression()

In [ ]:
# выводим показатели качества модели на тестовой выборке
predictions = lr.predict(X_test[most_important_features])
print('MAE: ', mean_absolute_error(y_test, predictions))
print('MAPE :', mean_absolute_percentage_error(y_test, predictions))

MAE:  0.36034819444685423
MAPE : 0.023531624562503493


In [ ]:
# Улучшаем метрики с помощью feature engineering, добавим новые признаки год и месяц

X_train['year'] = X_train['timestamp'].apply(lambda x: x[:4]).astype(int)
X_train['month'] = X_train['timestamp'].apply(lambda x: x[5:7]).astype(int)

X_test['year'] = X_test['timestamp'].apply(lambda x: x[:4]).astype(int)
X_test['month'] = X_test['timestamp'].apply(lambda x: x[5:7]).astype(int)

In [ ]:
# выводим показатели качества модели на обучающей выборке
lr = LinearRegression()
print('MAE: ',
      np.array(cross_val_score(lr, X_train[most_important_features + ['year', 'month']], 
                               y_train, scoring='neg_mean_absolute_error', cv=5)).mean())
print('MAPE: ', 
      np.array(cross_val_score(lr, X_train[most_important_features + ['year', 'month']],
                               y_train, scoring='neg_mean_absolute_percentage_error', cv=5)).mean())
lr.fit(X_train[most_important_features + ['year', 'month']], y_train)

MAE:  -0.350094292200105
MAPE:  -0.022902729632710978


LinearRegression()

In [ ]:
# выводим показатели качества модели на тестовой выборке
# новые признаки незначительно улучшили нашу модель
predictions = lr.predict(X_test[most_important_features + ['year', 'month']])
print('MAE: ', mean_absolute_error(y_test, predictions))
print('MAPE :', mean_absolute_percentage_error(y_test, predictions))

MAE:  0.3466520065538308
MAPE : 0.0226393505303053


Используя методы feature engineering получаем высокое качество модели